This is where combined_tweets.csv is made

In [3]:
#insert your credentials here
import os
from dotenv import load_dotenv
import dataset
import json
import csv
import tweepy
from sqlalchemy.exc import ProgrammingError
import pandas as pd

In [4]:
load_dotenv()

#db = dataset.connect(os.getenv("DB_URL"))

CONSUMER_KEY = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET = os.getenv('CONSUMER_SECRET')
ACCESS_KEY = os.getenv('ACCESS_KEY')
ACCESS_SECRET = os.getenv('ACCESS_SECRET')

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)

In [12]:
db = pd.DataFrame(columns=['tweets'])

In [9]:
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        filter_words = ["police", "officer", "cop"]
        conditions = (not 'RT @' in status.text) and any(word in status.text for word in filter_words)
        if conditions:
            description = status.user.description
            loc = status.user.location
            text = status.text
            coords = status.coordinates
            geo = status.geo
            name = status.user.screen_name
            user_created = status.user.created_at
            id_str = status.id_str
            created = status.created_at
            source = status.user.url
            language = status.lang

            if geo is not None:
                geo = json.dumps(geo)

            if coords is not None:
                coords = json.dumps(coords)

            table = db["tweets"]
            try:
                table.insert(dict(
                    user_description=description,
                    user_location=loc,
                    coordinates=coords,
                    text=text,
                    geo=geo,
                    user_name=name,
                    user_created=user_created,
                    id_str=id_str,
                    created=created,
                    source = source,
                    language = language,
                    ))
            except ProgrammingError as err:
                print(err)

    def on_error(self, status_code):
        if status_code == 420:
            #return False in on_data disconnects the stream
            return False

In [ ]:
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=["police", "cop", "officer"])

In [ ]:
df_raw = pd.DataFrame(db['tweets'])
df_raw = df_raw[df_raw['language'].isin(["en", "und"]) ]

In [ ]:
df = df_raw[["id_str", "text"]]
df['reddit'] = 0
df.rename(columns={'id_str': 'ids'}, inplace=True)

In [ ]:
df

In [ ]:
df_reddit = pd.read_csv('reddit_tweets.csv')

In [ ]:
df_combined = pd.concat([df_reddit, df])
df_combined

In [ ]:
# created CSV
df_combined.to_csv("combined_tweets.csv", index=False)